# Math 4 - Linear Algebra: The Inverse of a Matrix

In [1]:
import numpy as np
import scipy.linalg

his notebook is part of a series of math topics. You can find the previous chapter **Linear Algebra: Gaussian Elimination** [here](https://github.com/jaunerc/cs-math/blob/master/math-3-linear-algebra-gaussian-elimination.ipynb)

## 1. The inverse of a matrix
Let's start with a simple case. The (multiplicative) inverse of every real number $r$ (except zero) is just $\frac{1}{r}$ or in a different notation $r^{-1}$. This is because the product of a real number with its inverse must be equal to $1$. So we have $r^{-1} * r = \frac{1}{r} * r = 1$.